In [1]:
## Add Required Dependencies
%pip install sacrebleu
%pip install rouge_score
%pip install py7zr
import matplotlib.pyplot as plt
from datasets import load_dataset
from datasets import load_metric
from transformers import pipeline, set_seed
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd
import numpy as np
from tenacity import AttemptManager
from tqdm import tqdm
import torch
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
device = "cuda" if torch.cuda.is_available() else "cpu"


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Introduction

In this project, we build a encoder-decoder model from scratch to condense dialogues between several people into a crisp summary. Let's first look at one of the most famous datasets for summarization training: the CNN/DailyMail corpus.

## The CNN/DailyMail Dataset

This datasets consists of around 300,00 pairs of news articles and their corresponding summaries, composed from the bullet points that CNN and the DailyMail attach to their articles. An important aspect of the dataset is that the summaries are _abstractive_ and not _extractive_, which means that they consist of new sentences instead of simple excerts. The dataset is available on the hub, and this chapter uses version 3.0.0, which is an nonanonymized version set up for summarization. We can select versions in a similar manner as splits, with a _version_ keyword.

In [2]:
dataset = load_dataset("cnn_dailymail", version="3.0.0")
print(f"Features: {dataset['train'].column_names}")

Using custom data configuration default
Reusing dataset cnn_dailymail (C:\Users\lwhieldon\.cache\huggingface\datasets\cnn_dailymail\default\3.0.0\1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

Features: ['article', 'highlights', 'id']


The dataset has 3 columns: article, which contains the news articles, highlights with the summaries, and id to uniquely identify each article. Let's look at an excerpt from an article:

In [3]:
sample = dataset["train"][1]
print(f"""Article (excerpt of 500 characters, total length: {len(sample["article"])})):""")
print(sample["article"][:500])
print(f'\nSummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])

Article (excerpt of 500 characters, total length: 4051)):
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

Summary (length: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


Long articles pose a challenge to most transformer models since the context size is usually limited to 1,000 tokens or so, which is equivalent to a few paragraphs of text [insert citation here]. The standard, yet crude way to deal with this for summarization is to simply truncate the texts beyond the model's context size. Obviously, there could be important information for the summary toward the end of the text, but for now we need to live with this limitation of the model architectures.

## Text Summarization Pipelines

Let's see how  for summarization perform by first looking qualitatively at the outputs for the preceding example. Although the model architectures we will be exploring have varying maximum input sizes, let's retrict the input text to 2,000 characters to have the same input for all models and thus make the outputs for comparable:

In [8]:
sample_text = dataset["train"][1]["article"][:2000]
#We'll collect the generated summaries of each model in a dictionary 
summaries = {}

A convention is summarization is to separate the summary sentences by a newline. We could add a newline token after each fulle stop, but this simple heuristic would fail for string like "U.S." or "U.N". The Natural Lanugage Toolkit (NLTK) package includes a more sophisticated algorithm that can differentiate the end of a sentence from punctuation that occurs in abbreviations:

In [5]:
nltk.download("punkt")

string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lwhieldon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['The U.S. are a country.', 'The U.N. is an organization.']

### Summarization Baseline

A common baseline for summarizing news articles is to simply take the first three sentences of the article. With NLTK's sentence tokenizer, we can easily implement such a baseline:

In [10]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

summaries["baseline"] = three_sentence_summary(sample_text)


In [13]:
summaries

{'baseline': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."',
 'pegasus': 'Mentally ill inmates in Miami are housed on the "forgotten floor"<n>The ninth floor is where they\'re held until they\'re ready to appear in court.\nMost often, they face drug charges or charges of assaulting an officer.\nThey end up on the ninth floor severely mentally disturbed .'}

### PEGASUS

PEGASUS is an encoder-decoder transformer. As shown in figure below, its pretraining objective is to predict masked sentences in multisentence texts. The authors of [insert PEGASUS authors here] argue that the closer the pretraining objective is to the downstream task, the more effective it is. With the aim of finding a pretraining objective that is closer to summarization than general lanugage modeling, they automatically identified, in a very large corpus, sentences containing most of the content of their surrounding paragraphs (using summarization evaluation metrics as a heuristic for content overlap) and pretrained the PEGASUS model to reconstruct these sentences, thereby obtaining a state-of-the-art model for text summarization.

![PegasusArchitecture](PegasusArchitecture.png)

This model has a special token for newlines, which is why we don't need the sent_tokenize() function:

In [11]:
pipe = pipeline("summarization",model="google/pegasus-cnn_dailymail")
pipe_out = pipe(sample_text)
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>",".\n")

## Comparing Different Summaries

Now that we have generated summaries with four different models, let's compare the results. Keep in mind that one model has not been trained on the dataset at all (GPT-2), one model has been fine-tuned on this task among others (T5), and two models have exclusively been fine-tuned on this task (BART and PEGASUS). Let's have a look at the summaries these models have generated:

In [12]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .

BASELINE
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."

PEGASUS
Mentally ill inmates in Miami are housed on the "forgotten floor"<n>The ninth floor is where they're held until they're ready to appear in court.
Most often, they face drug charges or charges of assaulting an office

But how do you score the standard The standard metrics that we've seen, like accuracy, recall, and precision, are not easy to apply to this task. For each "gold standard" summary written by a human, dozens of other summaries with synonyms, paraphrases, or a slightly different way of formulating the facts could be just as acceptable.

Let's look at some common metrics that have been developed for measuring the quality of generated text.

## Measuring the Quality of Generated Text

Good evaluation metrics are important, since we use them to measure the performance of models not only when we train them but also later, in production. If we have bad metrics we might be blind to model degradation, and if they are misaligneed with the business goals we might not create any value.

Measuring performance on text generation tasks is not easy as with standard classification tasks such as sentiment analysis or name entity recognition. Take the example of translation; given a sentence like "I love dogs!" in English and translating it to Spanish there can be mutliple valid possibilities, like “¡Me encantan los perros!” or “¡Me gustan los perros!”. Simply checking for an exact match to a reference translation is not optimal; even humans would fare badly on such a metric because we all write text slightly differently for each other (and even from ourselves, depending on the time of day or year!). Fortunately, there are alternatives.

Two of the most common metrics used to evaluate generated text are BLEU and ROUGE. Let's take a look at how they're defined.

### BLEU

The idea of BLEU is simple: Instead of looking at how many of the tokens in the generated texts are perfectly aligned with the reference text tokens, we look at work or n-grams (Journal here describes n-grams very well: https://web.stanford.edu/~jurafsky/slp3/3.pdf). BLEU is a precision-based metric, which means that when we compare two texts we count the number of words in the generation that occur in the reference and divide it by the length of the generation.

However, there is an issue with this vanilla precision. Assume the generated text just repeats the same work over and over again, and this word also appears in the reference. If it is repeated as many times as the length of the reference text, then we get perfect precision! For this reason, the authors of BLEU paper introduced a slight modification: a word is only counted as many times as it occurs in the reference. To illustrate this point, suppose we have the reference text "the cat is on the mat" and the generated text "the the the the the the".

From this simple sample, we can calculate the precision values as follows:

$$P_{vanilla} = \frac{6}{6}$$

$$P_{vanilla} = \frac{2}{6}$$

and we can see that the simple correction has produced a much more reasonable value. Now let's extend this by not only looking at single words, but _n-grams_ as well. Let's assumed we have one generated sentence, $snt$, that we want to compare against a reference sentence,$snt'$. We extract all possible n-grams of degree n and do the accounting to get the precision, $P^{n}$:

$$P^{n} = \frac{\sum_{n-gram\in snt'}Count_{clip}(n-gram)}{\sum_{n-gram\in snt}Count(n-gram)}$$

In order to avoid rewarding repetitive generations, the count in the numerator is clipped. What this means is that the occurrence count of an _n-gram_ is capped at how many times it appears in the reference sentencne. Also note that the defition of a sentence is not very strict in this equation, and if you had a generated text spanning multiple sentences you would treat it as one sentence.

In general, we have more than one sample in the test set we want to evaluate, so we need to slightly extend the equation by summing over all samples in the corpus C:

$$P^{n} = \frac{\sum_{ snt \in C}\sum_{n-gram\in snt'}Count_{clip}(n-gram)}{\sum_{snt\in C}\sum_{n-gram\in snt}Count(n-gram)}$$

We're almost there. Since we are not look at recall, all generated sequences that are short but precise have a benefit compared to sentences that are longer. Therefore, the precision score favors short generations. To compensate for that the authors of BLEU introduced an additional term, the _brevity penalty_:

$$BR = min ( 1, e^{1-\ell_{ref} / \ell_{gen}} )$$

By taking the minimum, we ensure that this penalty never exceeds 1 and the exponential term becomes exponentially small when the length of the generated $l_{gen}$ is smaller thatn the reference text $l_{ref}$. At this point you might ask, why don't we just use something like an $F_{1}$-score to account for recall as well? The answer is that often in translation datasets there are multiple reference sentences instead of just one, so if we also measured recall we would incentivize translations that used all the words from all the references. Therefore, it's preferable to look for high precision in the translation and make sure the translation and reference have a similar length. 

Finally, we can put everything together and get the equation for the BLEU score.

$$BLEU-N = BR \times ( { \prod_{n-1}^{N} p_{n} })^{1/N}$$

The last term is the geometric mean of the modified precision up to n-gram N. In practice, BLEU-4 score is often reported. However, you can probably already see that this metric has many limitations; for instance, it doesn't take synonyms into account, and many steps in the derivation seem like ad hoc and rather fragile heuristics. Here's a link to a good medium article that discusses the limitations: https://towardsdatascience.com/evaluating-text-output-in-nlp-bleu-at-your-own-risk-e8609665a213

In general, the field of text generation is still looking for better evaluation metrics, and finding ways to overcome the limits of metrics like BLEU is an active area of research. Another weakness of the BLEU metric is that is expects the text to already be tokenized. This can lead to varying results if the exact same method for text tokenization is not used. The SecreBLEU metric addresses this issue  by internalizing the token step; for this reason, it is the preferred metric for benchmarking. We can load the metrics to calcate the score below:

In [14]:
bleu_metric = load_metric("sacrebleu")

The bleu_metric object is an instance of the Metric class, and works like an aggregator: you can add single instances with add() or whole batches via add_batch(). Once you have added all the samples you need to evaluate, you then call compute() and the metric is calculated. This returns a dictionary with several values, such as the precision for each n-gram, the length penalty, as well as the final BLEU score.

In [15]:
bleu_metric.add(prediction="the the the the the the",reference=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p,2) for p in results["precisions"]]
pd.DataFrame.from_dict(results,orient="index",columns=["Value"])

,Value
score,0.0
counts,"[2, 0, 0, 0]"
totals,"[6, 5, 4, 3]"
precisions,"[33.33, 0.0, 0.0, 0.0]"
bp,1.0
sys_len,6
ref_len,6


We can see the precision of the 1-gram is indeed 2/6, whereas the precisions for the 2/3/4-grams are all 0. This means the geometric mean is zero, and thus also BLEU score. Let's look at another example where the prediction is almost correct:

In [16]:
bleu_metric.add(prediction="the cat is on mat", reference=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor",smooth_value=0)
results["precisions"]=[np.round(p,2) for p in results["precisions"]]
pd.DataFrame.from_dict(results,orient="index",columns=["Value"])

,Value
score,57.893007
counts,"[5, 3, 2, 1]"
totals,"[5, 4, 3, 2]"
precisions,"[100.0, 75.0, 66.67, 50.0]"
bp,0.818731
sys_len,5
ref_len,6


We observe that the precision scores are much better. The 1-grams in the prediction all match, and only in the precision scores do we see that something is off. For the 4-gram there are only two candidates, ["the","cat","is","on"] and ["cat","is","on","mat"], where the last one does not match, hence the precision of 0.5.

The BLEU score is widely used for evaluating text, especially in machine translatiion, since precise translations are usually favored over translations that include all possible and appropriate words.

ROUGE score is usually used in summarization techniques, where we want all the important information in the generated text, so we favor high recall.

### ROUGE

The ROUGE score was specifically developed for applications like summarization where high recall is more important than just precision. The approach is very similar to the BLEU score in that we look at different n-grams and compare their occurrences in the generated text and the reference texts. The difference is that with ROUGE we check how many n-grams in the reference text also occur in the generated text. For BLEU we look at how many n-grams in the generated text appear in the reference, so we can reuse the precision formula with the minor modification that we coun the (unclipped) occurrence of reference n-grams in the generated text in the denominator:

$$ROUGE-N = \frac{\sum_{ snt' \in C}\sum_{n-gram\in snt'}Count_{match}(n-gram)}{\sum_{snt'\in C}\sum_{n-gram\in snt'}Count(n-gram)}$$

This was the original proposal for ROUGE. Subsequently, researchers have found that fully removing precision can have strong negative effects. Going back to the BLEU formula without the clipped counting, we can measure precision as well, and we can then combine both precision and recall ROUGE scores in the harmonic mean to get an $F^1$-score. This score is the metric that is nowadays commonly reported for ROUGE.

There is a separate score in ROUGE to measure the __longest common substring__ (LCS), called ROUGE-L. The LCS can be calculated for any pair of strings. For example, the LCS for "abab" and "abc" would be "ab", and its length would be 2. If we want to compare this value between two samples we need to somehow normalize it because otherwise a longer text would be at an advantage. To achieve this, the inventor of ROUGE campe up with an F-Score-like scheme where the LCS is normalized with the length of the reference and generated text, then the two normalized scores are mixed together

$$R_{LCS} = \frac{LCS(X,Y)}{m}$$

$$P_{LCS} = \frac{LCS(X,Y)}{n}$$

$$F_{LCS} = \frac{(1 + \beta^2) R_{LCS}P_{LCS}}{R_{LCS} + \beta P_{LCS}}, where \beta = P_{LCS}/R_{LCS}$$

The way the LCS score is properly normalized and can be compared across samples. In 🤗 Datasets implementation, two variations of ROUGE are calculated: one calculates the score per sentence and averages it for the summaries (ROUGE-L), and the other calculates it directly over the whole summary (ROUGE-Lsum).

We can load the metric as follows:



In [17]:
rouge_metric = load_metric("rouge")

We already generated a set of summaries with GPT-2 and the other models, and now we have a metric to compare the summaries systematically. Let's apply the ROUGE score to all the summaries generated by the models.

In [18]:
reference = dataset["train"][1]["highlights"]
records = []
rouge_names = ["rouge1","rouge2","rougeL","rougeLsum"]

for model_name in summaries:
    rouge_metric.add(prediction=summaries[model_name],reference=reference)
    score = rouge_metric.compute()
    rouge_dict = dict((rn,score[rn].mid.fmeasure) for rn in rouge_names)
    records.append(rouge_dict)

pd.DataFrame.from_records(records,index=summaries.keys())  

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.365079,0.145161,0.206349,0.285714
pegasus,0.323232,0.206186,0.282828,0.323232


These results are obviously not very reliable as we only looked at a single sample, but we can compare the quality of the summary for that one example. The table confirms out observation that ofthe models we considered, GPT-2 performs worst. This is not surprising since it is the only model of the group that was not explicity trained to summarize. Is it striking, however, that the simple first-three-sentence baseline doesn't fare too poorly compared to the transformer models that have on the order of a billion parameters! PEGASUS and BART are the best models overll (higher ROUGE scores are better), but T5 is slightly better on ROUGE-1 and the LCS scores. These results place T5 and PEGASUS as the best models, but again these results should be treated with caustion as we only evaluated the models on a single example. Looking at the results in the PEGASUS paper, we would expect the PEGASUS to outperform T5 on the CNN/DailyMail dataset.

Let's see if we can reproduce those results with PEGASUS.

## Evaluating Pegasus on the CNN/Daily Mail Dataset

We now have all the pieces in place to evaluate the model properly: we have a dataset with a test set from CNN/DailyMail, we have a metric with ROUGE, and we have a summarization model. We just need to put the pieces together. Let's first evaluate the performance of the three-sentence baseline:

In [19]:
def evaluate_summaries_baseline(dataset, metric, column_text="article",column_summary="highlights"):
    summaries = [three_sentence_summary(text) for text in dataset[column_text]]
    metric.add_batch(predictions=summaries,references=dataset[column_summary])
    score = metric.compute()
    return score

Now we'll apply the function to a subset of data. Since the test fraction of the CNN/DailyMail datasets consists of roughtly 10,000 samples, generating summaries for all these articles takes a lot of time. Every generated token requires a forward pass through the model; generating just 100 tokens for each sample will thus require 1 million forward passes, and if we use beam search this number is multiplied by the number of beams. For the purpose of keeping the calculations relatively fast, we'll subsample the test set and run the evaluation on 1,000 samples instead. This should give us much more stable score estimation while completing in less than one hour on a single GPU for the PEGASUS model:

In [20]:
test_sampled = dataset["test"].shuffle(seed=42).select(range(1000))

score = evaluate_summaries_baseline(test_sampled, rouge_metric)
rouge_dict = dict((rn,score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict, orient="index",columns=["baseline"]).T

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.388891,0.171452,0.245285,0.35433


The scores are mostly word than on the previous example, but still better than those achieved by GPT-2! Now let's implement the same evaluation functionn for evaluating the PEGASUS model:

In [21]:
def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements),batch_size):
        yield list_of_elements[i : i + batch_size]

def evaluate_summaries_pegasus(dataset,
                                metric,
                                model,
                                tokenizer,
                                batch_size=16,
                                device=device,
                                column_text = "article",
                                column_summary = "highlights"):
    article_batches = list(chunks(dataset[column_text],batch_size))
    target_batches = list(chunks(dataset[column_summary],batch_size))
    
    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length",return_tensors="pt")
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                   attention_mask=inputs["attention_mask"].to(device),
                                   length_penalty=0.8, num_beams=8, max_length=128)
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,clean_up_tokenization_space=True) for s in summaries]
        decoded_summaries = [d.replace("<n>"," ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()

    return score

Let's unpack this evaluation code a bit. First we split the dataset into smaller batches that we can process simultaneously. Then for each batch we tokenize the input articles and feed them to the generate() function to produce the summaries using beam search. We use the same generation parameters as proposed in the paper. The new parameter for length penalty ensures that the model does not generate sequences that are too long. Finally, we decode the generated texts, replace the <n> token, and add the decoded texts with the references to the metric. At the end, we compute and return the ROUGE scores. Let's now load the model again with the AutoModelForSeq2SeqLM class, used for seq2seq generation tasks, and evaluate it:

In [22]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
score = evaluate_summaries_pegasus(test_sampled, rouge_metric,model,tokenizer,batch_size=8)
rouge_dict = dict((rn,score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["pegasus"])

  2%|▏         | 3/125 [15:48<10:40:00, 314.76s/it]

These numbers are very close to the published results. One thing to note here is that the loss and per-token accuracy are decoupled to some degree from the ROUGE scores. The loss is indepdendent of the decoding strategy, whereas the ROUGE score is strongly coupled.

Since ROUGE and BLEU correlate better with human judgement than loss or accuracy, we should focus on them and carefully explore and choose the decoding strategy when building text generation models. These metrics are far from perfect, however, and one should also consider human judgement as well.

Now that we're equipped with an evaluation function, it's time to train our own model for summarization.

## Training a Summarization Model

We've worked through a lot of details on text summarization and evaluation, so let's put this to use to train a custom text summarization model! For our application, we'll use the SAMSum dataset, developed by Samsung, which consists of a collection of dialogues along with brief summaries. In an enterprise setting, these dialogues might represent the interactions between a customer and the support center, so generating accurate summaries can help improve customer service and detect common patterns among customer requests. Let's load it and look at an example:

In [ ]:
dataset_samsum = load_dataset("samsum")
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")
print(dataset_samsum["test"][0]["dialogue"])
print("\nSummary:")
print(dataset_samsum["test"][0]["summary"])

The dialogues look like what you would expect from a chat via SMS or WhatsApp, including emojis and placeholders for GIFs. The dialogue field contains the full text and the summary the summarized dialogue. Could a model that was fine-tuned on the CNN/DailyMail dataset deal with that? Let's find out!

### Evaluating PEGASUS on SAMSum

First we'll run the same summarization pipeline with PEGASUS to see what the output looks like. We can reuse the code we use for the CNN/DailyMail summary generation:

In [ ]:
pipe_out = pipe(dataset_samsum["test"][0]["dialogue"])
print("Summary: ")
print(pipe_out[0]["summary_text"].replace(" .<n>",".\n"))

We can see that the model mostly tries to summarize by extracting the key sentences from the dialogue. This probably worked relatively well on the CNN/DailyMail dataset, but the summaries in SAMSum are more abstract. Let's confirm this by running the full ROUGE evaluation on the test set:

In [ ]:
score = evaluate_summaries_pegasus(dataset_samsum["test"],
                                    rouge_metric, 
                                    model, 
                                    tokenizer, 
                                    column_text="dialogue",
                                    column_summary="summary",
                                    batch_size=8)
rouge_dict = dict((rn,score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["pegasus"])

The results aren't great but this is not unexpected since we've moved quite a bit away from the CNN/DailyMail data distribution. Nevertheless, setting up the evaluation pipeline before training has two advantages: we can directy measure the success of training with the metric and we have a good baseline. Fine-tuning the model on our dataset should result in an immediate improvement in the ROUGE metric, and if that is not the case we'll know something is wrong with our training loop.

### Fine-Tuning PEGASUS

Before we process the data for training, let's have a quick look at the length distribution of the input and outputs:

In [ ]:
d_len = [len(tokenizer.encode(s)) for s in dataset_samsum["train"]["dialogue"]]
s_len = [len(tokenizer.encode(s)) for s in dataset_samsum["train"]["summary"]]

fig,axes = plt.subplots(1,2,figsize=(10, 3.5), sharey=True)
axes[0].hist(d_len, bins=20, color="C0",edgecolor = "C0")
axes[0].set_title("Dialogue Token Length")
axes[0].set_xlabel("Length")
axes[0].set_ylabel("Count")
axes[1].hist(s_len,bins=20,color="C0",edgecolor="C0")
axes[1].set_title("Summary Token Length")
axes[1].set_xlabel("Length")
plt.tight_layout()
plt.show()

We see that most dialogues are much shorter than the CNN/DailyMail articles, which is 100-200 tokens per dialogue. Similarly, summaries are much shorter, with around 20-40 tokens (the average length of a tweet).

Let's keep those observations in mind as we build the data collator for the Trainer. First we need to tokenize the dataset. For now, we'll set the maximum lengths to 1024 and 128 for the dialogues and summaries, respectively:

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["dialogue"],max_length=1024, truncation=True)
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch["summary"], max_length=128,truncation=True)
    return {"input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]}

dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True)

columns = ["input_ids","labels","attention_mask"]
dataset_samsum_pt.set_format(type="torch",columns=columns)

A new thing in the use of the tokenization step is the `tokenizer.as_target_tokenizer()` context. Some models require special tokens in the decoder inputs, so it's important to differentiate between the tokenization of encoder and decoder iputs. In the `with` statement (called a _context manager_), the tokenizer knows that it is tokenizing for the decoder and can process sequences accordingly.

Now we need to create the data collator. This function is called in the `Trainer` just before the batch is fed through the model. In most cases we can use the default collator, which collects all the tensors from the batch and simply stacks them. For the summarization task we need to not only stack the inputs but also prepare the targets on the decoder side. PEGASUS is an encoder-decoder transformer and thus has the classic seq2seq architecture. In a seq2seq setup, a common approach is to apply "teacher forcing" in the decoder. With this strategy, the decoder receives input tokens (like in decoder-only models such as GPT-2) that consists of the labels shifted by one in additiona to the encoder output; so, when making the prediction for the next token the decoder gets the ground truth shifted by one as an input.

We shift it by one so that the decoder only see the previous group truth labels and not the current or future ones. Shifting alone suffices since the decoder has masked self-attention that masks all inputs at present and in the future.

So, when we prepare our batch, we set up the decoder inputs by shifting the labels to the right by one. After that, we make sure the padding tokens in the labels are ignored by the loss function by setting them to -100. We actually don't have to do this manually, though, since the `DataCollatorForSeq2Seq` comes to the rescue and takes care of all these steps for us:



In [ ]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Then, as usual, we set up the `TrainingArguments` for training:

In [ ]:
training_args = TrainingArguments(output_dir='pegasus-samsum',
                                  num_train_epochs=1,
                                  warmup_steps=500,
                                  per_device_train_batch_size=1,
                                  per_device_eval_batch_size=1,
                                  weight_decay=0.01,
                                  logging_steps=10,
                                  push_to_hub=True,
                                  evaluation_strategy='steps',
                                  eval_steps=500,
                                  save_steps=1e6,
                                  gradient_accumulation_steps=16
                            )

One thing that is different from the previous settings is that new argument, `gradient_accumulation_steps`. Since the model is quite big, we had to set the batch size to 1. However, a batch size that is too small can hurt convergence. To resolve the issue, we can use a nifty technique called _gradient accummulation_. As the name suggests, instead of calculating the gradients of the full batch all at once, we make smaller batches and aggregate the gradients. When we have aggregated enough gradients, we run the optimization step. Naturally, this is a bit slower than doing it in one pass, but it saves us a lot of GPU memory.

Let's now make sure that we are logged in to Hugging Face so we can push the model to the Hub after training:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

We now have everything we need to initialize the trainer with the model, tokenizer, training arguments, and data collator, as well as the training and evaluation sets:

In [ ]:
trainer = Trainer(model=model,
                  args=training_args,
                  tokenizer=tokenizer,
                  data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])

We are ready for training. After training, we can directly run the evaluation function on the test set to see how well the model performs:

In [ ]:
trainer.train()
score = evaluate_summaries_pegasus(dataset_samsum["test"],
                                    rouge_metric,
                                    trainer.model,
                                    tokenizer,
                                    batch_size=2,
                                    column_text="dialogue",
                                    column_summary="summary")

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f"pegasus"])

We see that the ROUGE scores improved considerably over the model without fine-tuning, so even though the previous model was also trained for summarization, it was not well adapted for the new domain. Let's push our model to the Hub:

In [ ]:
trainer.push_to_hub("Training complete!")

### Generating Dialogue Summaries

Looking at the losses and ROUGE score, it seems the model is showing a significant improvement over the original model trained on CNN/DailyMail only. Let's see what a summary generate on a sample from the test set looks like:

In [ ]:
gen_kwargs = {"length_penalty":0.8,"num_beams":8, "max_length":128}
sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]
pipe = pipeline("summarization",model="Lwhieldon/pegasus-samsum")

print("Dialogue")
print(sample_text)
print("\nReference Summary:")
print(reference)
print(pipe(sample_text,**gen_kwargs)[0]["summary_text"])

That looks much more like the reference summary. It seems the model has learned to synthesize the dialogue into a summary without just extracting passages. Now, the ultimate test: how well does the model work on custom input?

In [ ]:
custom_dialogue = """\
Thom: Hi guys, have you heard of transformers?
Lewis: Yes, I used them recently!
Leandro: Indeed, there is a great library by Hugging Face.
Thom: I know, I helped build it ;)
Lewis: Cool, maybe we should write a book about it. What do you think?
Leandro: Great idea, how hard can it be?!
Thom: I am in!
Lewis: Awesome, let's do it together!
"""

print(pipe(custom_dialogue,**gen_kwargs)[0]["summary_text"])

The generated summary of the custom dialogue makes sense. It summarizes well that all the people in the discussion want to write the book together and does not simply extract single sentences. 